In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

from sklearn.model_selection import GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression

In [2]:
df = pd.read_csv('train.csv')

In [3]:
df.head()

,ID,Company,Quarter,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange,Bond rating,Stock rating,Region,Industry,Sales
0,0,CMP01,Q1,2.02,7.71,0.05,-0.04,CCC,Buy,South,Metal Fabrication,1517.0
1,1,CMP01,Q2,2.01,4.10,0.03,0.00,CCC,Hold,South,Metal Fabrication,2968.0
2,2,CMP01,Q3,2.02,6.79,0.06,-0.02,CCC,Buy,South,Metal Fabrication,1497.0
3,3,CMP01,Q4,1.98,3.97,0.01,0.02,CCC,Buy,South,Metal Fabrication,2929.0
4,4,CMP01,Q5,1.96,7.41,-0.07,0.02,CCC,Buy,South,Metal Fabrication,1452.0


In [4]:
df.isna().sum()

ID                     0
Company                0
Quarter                0
QuickRatio             0
InventoryRatio       152
RevenueGrowth          0
MarketshareChange      0
Bond rating            0
Stock rating           0
Region                 0
Industry               0
Sales                150
dtype: int64

In [5]:
industry = pd.get_dummies(df['Industry'])[['Automobile', 'Infrastructure']]
region = pd.get_dummies(df['Region'])[['South', 'West', 'East']]
quarter = pd.get_dummies(df['Quarter'])[['Q1', 'Q2', 'Q3', 'Q4', 'Q5', 'Q6', 'Q7', 'Q8']]
company = pd.get_dummies(df['Company'])

stock_rating = {
    "Strong Sell": 1,
    "Sell": 2,
    "Hold": 3,
    "Buy": 4,
    "Strong Buy": 5
}

bond_rating = {
    "CCC": 1,
    "B": 2,
    "BB": 3,
    "BBB": 4,
    "A": 5,
    "AA": 6,
    "AAA": 7
}

In [6]:
ggs = df.groupby(["Bond rating", "Stock rating"]).median()[['InventoryRatio', 'Sales']]

df['InventoryRatio'].fillna(df[['Bond rating', 'Stock rating']].apply(lambda x: ggs['InventoryRatio'].loc[(x['Bond rating'], x['Stock rating'])], axis=1), inplace=True)
df['Sales'].fillna(df[['Bond rating', 'Stock rating']].apply(lambda x: ggs['Sales'].loc[(x['Bond rating'], x['Stock rating'])], axis=1), inplace=True)
df['InventoryRatio'].fillna(df['InventoryRatio'].median(), inplace=True)

/var/folders/06/qscsvt3d5z55r8bc2rcx__sw0000gn/T/ipykernel_24066/3482900235.py:1: FutureWarning: The default value of numeric_only in DataFrameGroupBy.median is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.
  ggs = df.groupby(["Bond rating", "Stock rating"]).median()[['InventoryRatio', 'Sales']]


In [7]:
df['Bond rating'] = df['Bond rating'].apply(lambda x: bond_rating[x])
df['Stock rating'] = df['Stock rating'].apply(lambda x: stock_rating[x])

In [8]:
df.drop(['ID', 'Company', 'Region', 'Industry', 'Quarter'], axis=1, inplace=True)
df = pd.concat([company, df, industry, region, quarter], axis=1)
df.head()

,CMP01,CMP02,CMP03,CMP04,CMP05,CMP06,CMP07,CMP08,CMP09,CMP10,...,West,East,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8
0,1,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
2,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
3,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
4,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0


In [9]:
X_train, X_test, y_train, y_test = train_test_split(df.drop(['Sales'], axis=1), df['Sales'], test_size=0.2)

# Hyperparameter Tuning for all algos

In [21]:
from sklearn.model_selection import GridSearchCV

rf = RandomForestRegressor()

param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

grid_search = GridSearchCV(estimator = rf, param_grid = param_grid,
                          cv = 5, n_jobs = -1, verbose = 2)

# Perform grid search on training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# You can use the best model obtained from grid search for predictions
best_rf = grid_search.best_estimator_


Fitting 5 folds for each of 243 candidates, totalling 1215 fits
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=None,

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=No

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth

[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=auto, min_samples_leaf=4, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=10, max_features=

Best Parameters: {'max_depth': None, 'max_features': 'sqrt', 'min_samples_leaf': 1, 'min_samples_split': 10, 'n_estimators': 200}
Best Score: 0.41710085361417193


/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
405 fits failed out of a total of 1215.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
70 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  Fi

In [22]:
rf = RandomForestRegressor(max_depth=None, max_features='auto', min_samples_leaf=1, min_samples_split=10, n_estimators=100)
rf.fit(X_train, y_train)
y_pred = rf.predict(X_test)
mean_absolute_error(y_test, y_pred)

InvalidParameterError: The 'max_features' parameter of RandomForestRegressor must be an int in the range [1, inf), a float in the range (0.0, 1.0], a str among {'sqrt', 'log2'} or None. Got 'auto' instead.

[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=2, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300; total time=   0.0s
[CV] END max_depth=20, max_features=l

[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END max_depth=None, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END max_depth=Non

[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.2s
[CV] END max_depth=None, max_features=sqrt, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.1s
[CV] END max_dep

In [ ]:
svr = SVR()

# Define parameter grid for tuning
param_grid = {
    'kernel': ['linear', 'rbf'],
    'C': [0.1, 1, 10],
    'gamma': ['scale', 'auto']
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator = svr, param_grid = param_grid,
                          cv = 5, n_jobs = -1, verbose = 2)

# Perform grid search on training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# You can use the best model obtained from grid search for predictions
best_svr = grid_search.best_estimator_

Fitting 5 folds for each of 12 candidates, totalling 60 fits
Best Parameters: {'C': 10, 'gamma': 'scale', 'kernel': 'linear'}
Best Score: 0.0005677795761712456


In [11]:
svr = SVR(C=10, gamma='scale', kernel='linear')
svr.fit(X_train, y_train)
y_pred = svr.predict(X_test)
acc = mean_absolute_error(y_test, y_pred)
acc

1138.660240931205

In [ ]:
knn = KNeighborsRegressor()

# Define parameter grid for tuning
param_grid = {
    'n_neighbors': [3, 5, 7, 9],  # Number of neighbors to consider
    'weights': ['uniform', 'distance'],  # Weight function used in prediction
    'metric': ['euclidean', 'manhattan']  # Distance metric used for the tree
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=knn, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2)

# Perform grid search on training data
grid_search.fit(X_train, y_train)

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# You can use the best model obtained from grid search for predictions
best_knn = grid_search.best_estimator_

Fitting 5 folds for each of 16 candidates, totalling 80 fits
Best Parameters: {'metric': 'manhattan', 'n_neighbors': 7, 'weights': 'uniform'}
Best Score: 0.3170507042495273


In [12]:
knn = KNeighborsRegressor(n_neighbors=7, weights='uniform', metric='manhattan')
knn.fit(X_train, y_train)
y_pred = knn.predict(X_test)
acc = mean_absolute_error(y_test, y_pred)
acc

1018.5497354497353

In [22]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import GridSearchCV

# Assuming X_train and y_train are your training data
# Assuming you have defined parameter grid for tuning

# Create a GradientBoostingRegressor
gb_regressor = GradientBoostingRegressor()

# Define parameter grid for tuning
param_grid = {
    'n_estimators': [100, 200, 300],
    'learning_rate': [0.01, 0.05, 0.1],
    'max_depth': [3, 5, 7],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['auto', 'sqrt', 'log2']
}

# Instantiate GridSearchCV
grid_search = GridSearchCV(estimator=gb_regressor, param_grid=param_grid,
                           cv=5, n_jobs=-1, verbose=2)

# Perform grid search on training data
grid_search.fit(df.drop(['Sales'], axis=1), df['Sales'])

# Get the best parameters and best score
best_params = grid_search.best_params_
best_score = grid_search.best_score_

print("Best Parameters:", best_params)
print("Best Score:", best_score)

# You can use the best model obtained from grid search for predictions
best_gb_regressor = grid_search.best_estimator_


Fitting 5 folds for each of 729 candidates, totalling 3645 fits
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=10

[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=300; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.

[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=2, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0

[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.0s
[CV] END learning_rate=0.01

[CV] END learning_rate=0.01, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=sqrt, min_samples_leaf=2, min_samples_split=5, n_estimators=300; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.1s
[CV] END learning_rate=0.01

...

[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01, max_depth=7, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200; total time=   0.1s
[CV] END learning_rate=0.01

[CV] END learning_rate=0.1, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.1, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.1, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.1, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.1, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.1, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.1, max_depth=3, max_features=auto, min_samples_leaf=1, min_samples_split=10, n_estimators=200; total time=   0.0s
[CV] END learning_rate=0.1,

[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.1s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=2, n_estimators=300; total time=   0.1s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.05, max_depth=7, max_features=log2, min_samples_leaf=1, min_samples_split=5, n_estimators=100; total time=   0.0s
[CV] END learning_rate=0.05

Best Parameters: {'learning_rate': 0.01, 'max_depth': 5, 'max_features': 'log2', 'min_samples_leaf': 1, 'min_samples_split': 5, 'n_estimators': 100}
Best Score: -0.05682580017402143


/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py:547: FitFailedWarning: 
1215 fits failed out of a total of 3645.
The score on these train-test partitions for these parameters will be set to nan.
If these failures are not expected, you can try to debug them by setting error_score='raise'.

Below are more details about the failures:
--------------------------------------------------------------------------------
833 fits failed with the following error:
Traceback (most recent call last):
  File "/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/model_selection/_validation.py", line 895, in _fit_and_score
    estimator.fit(X_train, y_train, **fit_params)
  File "/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 1467, in wrapper
    estimator._validate_params()
  File "/Users/preethi/anaconda3/lib/python3.11/site-packages/sklearn/base.py", line 666, in _validate_params
    validate_parameter_constraints(
  

In [23]:
from sklearn.ensemble import GradientBoostingRegressor

gbr = GradientBoostingRegressor(learning_rate=0.1, max_depth=3, max_features='log2', \
                                min_samples_leaf=1, min_samples_split=5, n_estimators=300)
gbr.fit(X_train, y_train)
y_pred = gbr.predict(X_test)
acc = mean_absolute_error(y_test, y_pred)
acc

937.1382944497062

## training on entire data

In [41]:
gbr = GradientBoostingRegressor(learning_rate=0.1, max_depth=3, max_features='log2', min_samples_leaf=1, min_samples_split=5, n_estimators=300)
gbr.fit(df.drop(['Sales'], axis=1), df['Sales'])

GradientBoostingRegressor(max_features='log2', min_samples_split=5,
                          n_estimators=300)

# `test.csv` Preprocessing

In [59]:
test = pd.read_csv('test.csv')
test.head()

,ID,Company,Quarter,QuickRatio,InventoryRatio,RevenueGrowth,MarketshareChange,Bond rating,Stock rating,Region,Industry
0,7,CMP01,Q8,1.93,2.79,-0.03,-0.01,CCC,Buy,South,Metal Fabrication
1,8,CMP01,Q9,1.93,4.77,0.00,0.00,CCC,Buy,South,Metal Fabrication
2,16,CMP02,Q8,1.97,2.34,0.04,-0.03,A,Sell,West,Infrastructure
3,17,CMP02,Q9,1.93,4.14,0.04,0.01,BBB,Hold,West,Infrastructure
4,25,CMP03,Q8,0.67,NaN,-0.05,-0.01,BB,Buy,East,Infrastructure


In [43]:
test['Quarter'].value_counts()

Q8    75
Q9    75
Name: Quarter, dtype: int64

In [44]:
industry = pd.get_dummies(test['Industry'])[['Automobile', 'Infrastructure']]
region = pd.get_dummies(test['Region'])[['South', 'West', 'East']]
quarter = pd.get_dummies(test['Quarter'])[['Q8']]
company = pd.get_dummies(test['Company'])

stock_rating = {
    "Strong Sell": 1,
    "Sell": 2,
    "Hold": 3,
    "Buy": 4,
    "Strong Buy": 5
}

bond_rating = {
    "CCC": 1,
    "B": 2,
    "BB": 3,
    "BBB": 4,
    "A": 5,
    "AA": 6,
    "AAA": 7
}

In [45]:
zeros = pd.DataFrame({
    "Q1": [0]*150,
    "Q2": [0]*150,
    "Q3": [0]*150,
    "Q4": [0]*150,
    "Q5": [0]*150,
    "Q6": [0]*150,
    "Q7": [0]*150,
})
quarter = pd.concat([zeros, quarter], axis=1)

In [46]:
ggs = df.groupby(["Bond rating", "Stock rating"]).median()[['InventoryRatio', 'Sales']]

test['InventoryRatio'].fillna(df[['Bond rating', 'Stock rating']].apply(lambda x: ggs['InventoryRatio'].loc[(x['Bond rating'], x['Stock rating'])], axis=1), inplace=True)
test['InventoryRatio'].fillna(df['InventoryRatio'].median(), inplace=True)

In [47]:
test['Bond rating'] = test['Bond rating'].apply(lambda x: bond_rating[x])
test['Stock rating'] = test['Stock rating'].apply(lambda x: stock_rating[x])

In [48]:
test.drop(['Company', 'Region', 'Industry', 'Quarter'], axis=1, inplace=True)
test = pd.concat([company, test, industry, region, quarter], axis=1)
test.head()

,CMP01,CMP02,CMP03,CMP04,CMP05,CMP06,CMP07,CMP08,CMP09,CMP10,...,West,East,Q1,Q2,Q3,Q4,Q5,Q6,Q7,Q8
0,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,1
3,0,1,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
4,0,0,1,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,1


In [57]:
x = test.drop(['ID'], axis=1)
y_pred = gbr.predict(x)

In [60]:
ids = test['ID']
submissions = pd.DataFrame({
    "ID": ids,
    "Sales": y_pred
})

submissions

,ID,Sales
0,7,2481.517713
1,8,2227.010669
2,16,5348.907800
3,17,4408.897182
4,25,4381.555632
...,...,...
145,656,3372.892924
146,664,2486.888044
147,665,2888.464500
148,673,2451.466105


In [61]:
submissions.to_csv("submission_d.csv", index=False)